# Fine-tuning a Model for Masked Language Modeling (MLM) Exam

In this exam, you will be tasked with performing dataset preprocessing and fine-tuning a model for a masked language modeling task. Complete each step carefully according to the instructions provided.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `bert-base-uncased` for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/math_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

In [139]:
!pip install datasets

In [150]:
!pip install torch

In [149]:
from transformers import BertTokenizer, BertForMaskedLM
from datasets import load_dataset
from transformers import pipeline
from transformers import Trainer, TrainingArguments
from transformers import pipeline
import torch

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [145]:
data_set = load_dataset('CUTD/math_df', split='train[:70%]')

## Step 2: Load the Pretrained Model and Tokenizer

In [146]:
toke = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Use a pre-trained model and tokenizer for this task. Initialize both in this step.

In [147]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Step 3: Preprocess the Dataset

Define a preprocessing function that tokenizes the text data and prepares the inputs for the model. Ensure that you truncate the sequences to a maximum length of 512 tokens and pad them appropriately.

**Bonus**: If you performed more comprehensive preprocessing, such as removing links, converting text to lowercase, or applying additional preprocessing techniques.

In [148]:
def preprocess_data(examples):
    inputs = toke(examples['text'], truncation=True, padding='max_length', max_length=512, return_tensors='pt')

    inputs['labels'] = inputs['input_ids'].clone()

    probability_matrix = torch.full(inputs['input_ids'].shape, 0.15)
    special_tokens_mask = toke.get_special_tokens_mask(inputs['input_ids'], already_has_special_tokens=True)
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    inputs['labels'][~masked_indices] = -100


    inputs['input_ids'][masked_indices] = toke.convert_tokens_to_ids(toke.mask_token)

    return inputs

train_data = data_set.map(preprocess_data, batched=True, remove_columns=["text"])

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [6]:
def preprocess_data(examples):
    inputs = toke(examples['text'], truncation=True, padding='max_length', max_length=512, return_tensors='pt')
    inputs['labels'] = inputs['input_ids'].clone()
    return inputs

train_data = data_set.map(preprocess_data, batched=True, remove_columns=["text"])

In [7]:
df = data_set.to_pandas()
df.head()

,Unnamed: 0,text
0,0,A software engineer who disagrees with the est...
1,1,A German literature college student who is int...
2,2,An entrepreneur who is developing a mobile app...
3,3,A pragmatic politician always considering the ...
4,4,An experienced film director who recognizes th...


## Step 4: Define Training Arguments

Set up the training configuration, including parameters like learning rate, batch size, number of epochs, and weight decay.

In [8]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    num_train_epochs=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data

)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer.train()

Step,Training Loss
500,0.346100


TrainOutput(global_step=875, training_loss=0.1978852355820792, metrics={'train_runtime': 875.8772, 'train_samples_per_second': 7.992, 'train_steps_per_second': 0.999, 'total_flos': 1842433689600000.0, 'train_loss': 0.1978852355820792, 'epoch': 1.0})

## Step 5: Initialize the Trainer

Initialize the Trainer using the model, training arguments, and datasets (both training and evaluation).

## Step 6: Fine-tune the Model

Run the training process using the initialized Trainer to fine-tune the model on the masked language modeling task.

## Step 7: Inference

Use the fine-tuned model for inference. Create a pipeline for masked language modeling and test it with a sample sentence.

In [120]:
s1 = 'we are the best of ([MASK]) the best'
s3 = ''

In [121]:
m = pipeline('fill-mask', model="bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issu

In [122]:
before= m(s1, top_k=1)

In [131]:
before2= m(s3, top_k=1)

In [132]:
print(before)

[{'score': 0.17430149018764496, 'token': 2133, 'token_str': '...', 'sequence': 'we are the best of (... ) the best'}]


In [133]:
print(before2)

[{'score': 0.7576868534088135, 'token': 1012, 'token_str': '.', 'sequence': 't5 bootcam foucsing on technolagy.'}]


In [134]:
m2 = pipeline('fill-mask', model= model,tokenizer=toke)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [135]:
after= m2(s1, top_k=1)

In [136]:
after2= m2(s3, top_k=1)

In [137]:
print(after)

[{'score': 0.9866095185279846, 'token': 0, 'token_str': '[PAD]', 'sequence': 'we are the best of ( ) the best'}]


In [138]:
print(after2)

[{'score': 0.443438857793808, 'token': 1025, 'token_str': ';', 'sequence': 't5 bootcam foucsing on technolagy ;'}]
